## Prediction of selected physical parameters of over-contact data with simple combined NN model
In this Jupyter Notebook we will train NN model to predict selected physical parameter of over-contact binary systems. Content:
* Libraries, functions
* Data preparation
* Create architecture of NN model
* Evaluation of model
* Predictions
* Evaluation of predictions

## 1. Environment set-up
* Importing libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Input, Dense, LSTM, Dropout, Flatten
from keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

np.random.seed(1234)
pd.set_option('display.max_rows', None)

* Defining functions for noise generation, set-up of random sigma value generator.
* Inputs:
    * for function generate_observation_sigma(space_obs_frac=0.5) - space_obs_frac=0.5
    * for function stochastic_noise_generator(curve) - vector of light curve

In [4]:
def generate_observation_sigma(space_obs_frac=0.5):
    """
    Draws a standard deviation of noise in light curve points from a "true" value provided in synthetic light curve.
    Noise sigma is drawn from bimodal distribution taking into account contributions from space based and earth based
    observations which have different levels of stochastic noise.

    :param space_obs_frac: ratio between earth based and space based observations
    :return: float; standard deviation of the light curve noise
    """
    earth_based_sigma = 4e-3
    space_based_sigma = 2e-4
    sigma = np.random.choice([earth_based_sigma, space_based_sigma], p=[1-space_obs_frac, space_obs_frac])
    return np.random.rayleigh(sigma)

def stochastic_noise_generator(curve):
    """
    Introduces gaussian noise into synthetic observation provided in `curve`.

    :param curve: numpy.array; normalized light curve
    :return: Tuple(numpy.array, float); normalized light curve with added noise, standard deviation of observations
    """
    sigma = generate_observation_sigma()
    return np.random.normal(curve, sigma), np.full(curve.shape, sigma)

## 2. Data loading
* Loading synthetic data from .pkl file

In [5]:
data = pd.read_pickle("overcontact_all_parameters.pkl").reset_index()

* Selecting random sample of data of size 300 000 records

In [6]:
data_sample = data.sample(n=300000)

## 3. Data preparation

* Create multi-dimensional array of vectors of light curves

In [8]:
X = []
for row in data_sample["curve"]:
    X.append(row)
X=np.array(X)

* Create array of features, which will model predict

In [9]:
y = np.array(data_sample[[
    "inclination",
    "mass_ratio",
    "primary__surface_potential",
    "secondary__surface_potential",
    "t1/t2"]])

* Defining MinMax scaler object
* Fitting scaler

In [10]:
scaler = MinMaxScaler()
y_minmax_scaled = scaler.fit_transform(y)
y_minmax_scaled[0]

array([0.79148929, 0.49494949, 0.53031622, 0.53031622, 0.        ])

* Splitting data into training and testing data sets in 80:20 ratio

In [11]:
X_train1, X_test, y_train1, y_test = train_test_split(X, y_minmax_scaled, test_size=0.2)

* Adding noise into training datasets (noise generated with functions defined earlier)

In [12]:
X_train_n = []
y_train_n = []
for i in range(len(X_train1)):
    for j in range(3):
        curve = stochastic_noise_generator(X_train1[i])
        X_train_n.append(curve[0])
        y_train_n.append(y_train1[i])
X_train_n = np.array(X_train_n)
y_train_n=np.array(y_train_n)

* Details about number of records in specific data sets

In [13]:
print("Number of records in dataset: ", len(data),
    "\nNumber of records in sample: ", len(X),
    "\nNumber of train data without noise: ", len(X_train1),
    "\nNumber of train data with noise: ", len(X_train_n),
    "\nNumber of test data without noise: ", len(X_test))

Number of records in dataset:  1212796 
Number of records in sample:  300000 
Number of train data without noise:  240000 
Number of train data with noise:  720000 
Number of test data without noise:  60000


## 4. Modeling

* Defining neural network model architecture
    * it is simple combined architecture with 1D CNN and recurrent LSTM layer
    * input shape of vector is  400x1, outpot is array 10x1 - 10 predicted physical parameters
    * model will be saved as *norm_overcontact_selection.hdf5* in *models* folder

In [14]:
inputs = Input(shape=(400, 1))
b = Conv1D(64, kernel_size = 3, padding = "valid")(inputs)
b = MaxPooling1D(2)(b)
b = Dropout(0.2)(b)
b = LSTM(64, return_sequences=True)(b)
b = Flatten()(b)
b = Dense(64, activation='relu')(b)
x = Dense(32, activation='relu')(b)
output = Dense(5, activation='linear')(x)
model = Model(inputs=inputs, outputs=output)
model.compile(loss='mse', optimizer='adam', metrics=["mae", "mape"])

saved_model = "models/norm_overcontact_selection.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor = 'val_mae', verbose = 1, save_best_only = True, mode = 'min')
early = EarlyStopping(monitor = "val_mae", mode = "min", patience = 25)
callbacks_list = [checkpoint, early]

print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 400, 1)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 398, 64)           256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 199, 64)           0         
_________________________________________________________________
dropout (Dropout)            (None, 199, 64)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 199, 64)           33024     
_________________________________________________________________
flatten (Flatten)            (None, 12736)             0         
_________________________________________________________________
dense (Dense)                (None, 64)               

* Model training
    * Model is trained for 10 epochs
    * For validation data set we selected 10% of training data

In [16]:
history = model.fit(X_train_n, y_train_n, validation_split = 0.1, epochs = 10, verbose = 1, callbacks = callbacks_list, batch_size = 64)

Epoch 1/10
10125/10125 [==============================] - ETA: 0s - loss: 0.0102 - mae: 0.0602 - mape: 7245895.0000
Epoch 00001: val_mae improved from inf to 0.04150, saving model to models\norm_overcontact_selection.hdf5
10125/10125 [==============================] - 1669s 165ms/step - loss: 0.0102 - mae: 0.0602 - mape: 7245895.0000 - val_loss: 0.0056 - val_mae: 0.0415 - val_mape: 5226817.5000
Epoch 2/10
10125/10125 [==============================] - ETA: 0s - loss: 0.0063 - mae: 0.0459 - mape: 5063508.5000
Epoch 00002: val_mae improved from 0.04150 to 0.03797, saving model to models\norm_overcontact_selection.hdf5
10125/10125 [==============================] - 1461s 144ms/step - loss: 0.0063 - mae: 0.0459 - mape: 5063508.5000 - val_loss: 0.0048 - val_mae: 0.0380 - val_mape: 4130595.2500
Epoch 3/10
10125/10125 [==============================] - ETA: 0s - loss: 0.0058 - mae: 0.0430 - mape: 4662523.5000
Epoch 00003: val_mae improved from 0.03797 to 0.03612, saving model to models\norm_o

* Loading of trained model

In [14]:
model = load_model("models/norm_overcontact_selection.hdf5")

## 5. Model evaluation

* Model evaluation on test data without added noise
* In the output we can see loss and MAE values

In [16]:
scores = model.evaluate(X_test, y_test)
print('Loss: {:.4f}, MAE: {:.4f}'.format(scores[0], scores[1]))

1875/1875 [==============================] - 82s 44ms/step - loss: 0.0038 - mean_absolute_error: 0.0322 - mean_absolute_percentage_error: 2902031.5000
Loss: 0.0038, MAE: 0.0322


* Adding random noise to test data

In [15]:
X_test_n = []
y_test_norm_n = []
for i in range(len(X_test)):
    for j in range(3):
        curve = stochastic_noise_generator(X_test[i])
        X_test_n.append(curve[0])
        y_test_norm_n.append(y_test[i])
        j += 1
X_test_n = np.array(X_test_n)
y_test_norm_n = np.array(y_test_norm_n)

* Model evaluation on test data with added noise
* In the output we can see loss and MAE values

In [17]:
scores_n = model.evaluate(X_test_n, y_test_norm_n)
print('Loss: {:.4f}, MAE: {:.4f}'.format(scores_n[0], scores_n[1]))

5625/5625 [==============================] - 514s 91ms/step - loss: 0.0040 - mean_absolute_error: 0.0336 - mean_absolute_percentage_error: 3030820.2500s - loss: 0.0040 - mean_absolute_er
Loss: 0.0040, MAE: 0.0336


## 6. Predictions on test data without noise

* Predictions on test data without noise
* Predictions are saved into *y_pred_norm* variable in the form of multi-dimensional array

In [18]:
y_pred_norm = model.predict(X_test)

* Since model is predicting normalized values, we need to denormalize array of predictions with use of inverse transformation

In [20]:
denorm = scaler.inverse_transform(y_pred_norm)
denorm[0]

array([1.00987  , 1.2422509, 3.814539 , 3.813457 , 1.0802377],
      dtype=float32)

* We create data frame of denormalized predictions with specific column names

In [21]:
denorm_pred_df = pd.DataFrame(denorm,
                           columns = [
                                "inclination",
                                "mass_ratio",
                                "primary__surface_potential",
                                "secondary__surface_potential",
                                "t1_t2"
                            ])
denorm_pred_df.head()

,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential,t1_t2
0,1.009870,1.242251,3.814539,3.813457,1.080238
1,1.434893,0.897172,3.358399,3.383627,1.033528
2,1.009152,0.991710,3.349905,3.248664,1.077547
3,1.329317,0.905684,3.490230,3.525199,1.076381
4,0.887441,0.914283,3.264172,3.233231,1.063057


* Average values for each predicted attribute calculated with *mean()* function

In [22]:
pred_mean = denorm_pred_df.mean(axis=0)
pred_mean

inclination                     1.207262
mass_ratio                      1.460771
primary__surface_potential      4.067463
secondary__surface_potential    4.059316
t1_t2                           1.045363
dtype: float32

* Data frame created from test datasets
* Average values for each attribute is calculated with *mean()* function

In [23]:
y_test_df = pd.DataFrame(y,
                        columns = [
                            "inclination",
                            "mass_ratio",
                            "primary__surface_potential",
                            "secondary__surface_potential",
                            "t1_t2"
                            ])
test_mean = y_test_df.mean(axis=0)
test_mean

inclination                     1.222845
mass_ratio                      1.422204
primary__surface_potential      3.987477
secondary__surface_potential    3.987477
t1_t2                           1.048233
dtype: float64

* Dataframe created for purpose to compare average true and predicted value, with Mean Average Error showed

In [24]:
eval_pred = pd.DataFrame({'attribute': test_mean.index,
            'avg_true': test_mean.values,
            'avg_pred': pred_mean.values,
            'MAE': abs(test_mean.values - pred_mean.values)})
eval_pred

,attribute,avg_true,avg_pred,MAE
0,inclination,1.222845,1.207262,0.015583
1,mass_ratio,1.422204,1.460771,0.038567
2,primary__surface_potential,3.987477,4.067463,0.079986
3,secondary__surface_potential,3.987477,4.059316,0.071839
4,t1_t2,1.048233,1.045363,0.002869


## 7. Prediction on test data with noise

* Prediction on test data with noise
* Predictions are save into *y_pred_norm_n* variable in the form of multi-dimensional array

In [25]:
y_pred_norm_n = model.predict(X_test_n)

* Since model is predicting normalized values, we need to denormalize array of predictions with use of inverse transformation

In [26]:
denorm_n = scaler.inverse_transform(y_pred_norm_n)
denorm_n[0]

array([1.0137995, 1.2389269, 3.8086123, 3.8100085, 1.0807337],
      dtype=float32)

* We create data frame of denormalized predictions with specific column names

In [27]:
denorm_pred_n_df = pd.DataFrame(denorm_n,
                            columns = [
                                "inclination",
                                "mass_ratio",
                                "primary__surface_potential",
                                "secondary__surface_potential",
                                "t1_t2"
                            ])
denorm_pred_n_df.head()

,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential,t1_t2
0,1.013800,1.238927,3.808612,3.810009,1.080734
1,1.009084,1.247791,3.822558,3.821386,1.079962
2,1.010949,1.236768,3.806676,3.805421,1.079818
3,1.434906,0.899390,3.361198,3.386827,1.033581
4,1.435269,0.872110,3.324525,3.346521,1.032596


* Average values for each predicted attribute calculated with *mean()* function

In [28]:
pred_n_mean = denorm_pred_n_df.mean(axis=0)
pred_n_mean

inclination                     1.207670
mass_ratio                      1.463276
primary__surface_potential      4.071081
secondary__surface_potential    4.063103
t1_t2                           1.045390
dtype: float32

* We create dataframe of denormalized test values
* Calculate average values of each attribute

In [30]:
y_test_n_denorm = scaler.inverse_transform(y_test_norm_n)
y_test_norm_n_df = pd.DataFrame(y_test_n_denorm,
                            columns = [
                            "inclination",
                            "mass_ratio",
                            "primary__surface_potential",
                            "secondary__surface_potential",
                            "t1_t2"
                            ])
test_mean_n = y_test_norm_n_df.mean(axis=0)
test_mean_n

inclination                     1.223116
mass_ratio                      1.434141
primary__surface_potential      4.002988
secondary__surface_potential    4.002988
t1_t2                           1.048256
dtype: float64

* Dataframe created for purpose to compare average true and predicted value, with Mean Average Error showed

In [31]:
eval_pred = pd.DataFrame({'attribute': test_mean_n.index,
            'avg_true': test_mean_n.values,
            'avg_pred': pred_n_mean.values,
            'MAE': abs(test_mean_n.values - pred_n_mean.values)})
eval_pred

,attribute,avg_true,avg_pred,MAE
0,inclination,1.223116,1.207670,0.015446
1,mass_ratio,1.434141,1.463276,0.029135
2,primary__surface_potential,4.002988,4.071081,0.068094
3,secondary__surface_potential,4.002988,4.063103,0.060115
4,t1_t2,1.048256,1.045390,0.002866
